## Post Call Analytics (PCA) Using Amazon Bedrock
Amazon Bedrock을 사용한 통화 후 분석 사용 사례에 대한 이 교육에 오신 것을 환영합니다.

기업이 다양한 채널을 통해 고객과 계속 상호 작용함에 따라 이러한 상호 작용을 분석하여 고객 행동 및 선호도에 대한 인사이트를 얻는 것이 점점 더 중요해지고 있습니다.<BR>
통화 후 분석은 통화 종료 후 고객 상호 작용을 분석하는 방법 중 하나입니다.<BR>
대규모 언어 모델을 사용하면 보다 정확한 감정 분석을 가능하게 하고, 특정 고객의 요구와 선호도를 파악하며, 전반적인 고객 경험을 개선함으로써 통화 후 분석의 효과를 크게 높일 수 있습니다.<BR>

이 샘플 노트북에서는 통화 후 분석에 Bedrock을 사용할 때 얻을 수 있는 다양한 이점과 기업이 현대 시장에서 경쟁 우위를 확보할 수 있는 방법을 다음 주제를 통해 살펴봅니다.<BR>

- 베드락에서 LLM 모델 선택(타이탄 텍스트 및 앤트로닉 클로드)
- 하나의 모델로 여러 PCA 작업 처리
- 긴 통화 기록 처리

# Environment Setup
샘플 코드를 실행하는 데 필요한 패키지를 설치 및 업그레이드합니다. <BR>
**참고: 업데이트된 패키지를 사용하려면 커널을 재시작해야 할 수 있습니다.**

# Import packages

In [ ]:
import boto3
import langchain
from termcolor import colored
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

# Load transcript files

In [ ]:
transcript_files = [
    "./call_transcripts/negative-refund-ko.txt",
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-ko.txt",
    "./call_transcripts/aws-ko-short.txt"
]

transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [ ]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

# Post Call Analysis

## Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

**Select nternal-use: True(aws 직원), False (고객)** <BR>
**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [ ]:
is_internal_use = <internal-use> ## if 고객: False, aws직원: True

In [ ]:
bedrock_region = <your region> ## "us-east-1" or "us-west-2"

In [ ]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock-runtime.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [ ]:
bedrock_config

In [ ]:
if is_internal_use:
    boto3_bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    boto3_bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"]
    ) 

In [ ]:
bedrock_models = {
    "Claude" : "anthropic.claude-v1",
    "TitanText": "amazon.titan-tg1-large", 
    "Claude-instant":"anthropic.claude-instant-v1",
    "Claude-V2" : "anthropic.claude-v2",
}

max_tokens = {
    "Claude" : 12000,
    "TitanText": 4096,
    "Claude-instant": 9000,
    "Claude-V2" : 12000,
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120, "Claude-V2" : 120}


In [ ]:
# Choose one of the bedrock model
model = "Claude-V2" # "Claude", "TitanText", "Claude-instant"
if model in ["Claude", "Claude-instant", "Claude-V2"]:
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=boto3_bedrock,
        model_kwargs={
            "max_tokens_to_sample":512,
            "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
            "temperature":0,
            "top_p":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
elif model == "TitanText":
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )

## Prompt Template
이 노트북에서는 네 가지 분석(**요약, 감성, 의도, 해결**)을 수행하게 되며, 각 분석에 대한 템플릿이 필요합니다.

In [ ]:
summary_template_ko = """\n\nHuman:

아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "{transcript}"

요약:

\n\nAssistant:"""

sentiment_template_ko = """\n\nHuman:

감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

고객 감성:

\n\nAssistant:"""

intent_template_ko = """\n\nHuman:

이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 

대화: "{transcript}"

고객 목적:

\n\nAssistant:"""

resolution_template_ko = """\n\nHuman:

이것은 해결 분류 프로그램입니다. 상담원이 문제를 해결한 방법은 다음과 같습니다. 
클래스 ["FULL_REFUND", "PARTIAL_REFUND", "QUESTION_ANSWERED", "UNRESOLVED"]. 대화를 다음 중 하나로 분류합니다. 
그리고 이 클래스 중 하나를 정확하게 분류하세요. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

상담원이 고객의 질문이나 문제를 어떻게 해결했는지 한 마디로 답하세요:

\n\nAssistant:"""

## Generate Analysis

In [ ]:
def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params
        
    print (colored(analysis_prompt, 'green'))
    response = llm(analysis_prompt)
    
    return response

In [ ]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
    "temperature":0,
    "top_p":0.9
}

### Summary

In [ ]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=summary_template_ko
)

print (res)

### Sentiment Analysis

In [ ]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=sentiment_template_ko
)

print (res)

### Intent Analysis

In [ ]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=intent_template_ko
)

print (res)

### Resolution Analysis

In [ ]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=resolution_template_ko
)

print (res)

## Handling long call transcripts
LLM의 인풋 토큰 한도를 초과하는 긴 문서를 처리하는 방법을 다룹니다. 

* Map Reduce
![nn](../../imgs/map_reduce.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

* Refine
![nn](../../imgs/refine.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [ ]:
stuff_prompt_template = """\n\nHuman:

다음 글을 간단하게 요약해 주세요.

글: {text}

요약:

\n\nAssistant:"""

chuck_prompt_template = """\n\nHuman:

다음 글을 간단하게 요약해 주세요.

글: {text}

요약:

\n\nAssistant:"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """\n\nHuman:

다음 글을 간단하게 요약해 주세요.

글: {text}

요약:

\n\nAssistant:"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [ ]:

'''
# summary_chain = load_summarize_chain(
#     llm=llm,
#     chain_type="map_reduce",
#     verbose=True
# ) # map_reduce, refine
# transcript = summary_chain(docs)
'''

def summary_chain_init(chain_type, llm):
    
    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [ ]:
def long_call_analysis(llm, transcript, params, chain_type="MAP_REDUCE", max_tokens=50):

    
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        
        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )
        
        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))
        
        return response["output_text"]
    
    else:
        
        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))
        
        response = llm(analysis_prompt)
        
        return response
        

In [ ]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
    "temperature":0,
    "top_p":0.9
}

In [ ]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=transcripts[4],
    params=PARAMS,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)

print ("Results: \n")
print (res)